# N-Gram Language Models
In this exercise, we will use n-gram language models to predict the probability of text, and generate it.

In [1]:
from nltk.corpus import gutenberg

First, we load Jane Austen's Emma from NLTK's gutenberg corpus that we also used in a previous exercise. Tokenize and lowercase this text such that we have a list of words.

In [2]:
import nltk

raw_text = gutenberg.raw('austen-emma.txt')

words = [word.lower() for word in nltk.word_tokenize(raw_text)]
print(words[:10])
print(len(words))

['[', 'emma', 'by', 'jane', 'austen', '1816', ']', 'volume', 'i', 'chapter']
191855


Write an n-gram language model class that takes the word list and a parameter `n` as inputs, where `n` is a positive integer larger than 1 that determines the `n` of the n-gram LM. The LM should build a dictionary of n-gram counts from the word list.

In [3]:
from collections import defaultdict

class NGramLanguageModel:
    def __init__(self, words, n):
        assert n > 1, "n needs to be a positive integer > 1"
        assert n <= len(words), "n can't be larger than the number of words"

        self.n = n
        self.ngrams = defaultdict(int) # use defaultdict to avoid having to always check if a key is in the dictionary

        for i in range(len(words) - n + 1): # i is the index of the first word in the ngram
            ngram = tuple(words[i:i+self.n])
            self.ngrams[ngram] += 1

            ngram_minus_one = ngram[:-1]
            self.ngrams[ngram_minus_one] += 1
        # also need to add the last n-1 words
        ngram_minus_one = ngram[1:]
        self.ngrams[ngram_minus_one] += 1

Now we "train" the n-gram LM by building the n-gram counts of the Emma novel. Use a low `n` (i.e. 2 or 3).

In [4]:
model = NGramLanguageModel(words, 3)

Let's add a method `log_probability` to the n-gram LM class that computes the probability of an input string. Since multiplying many probabilities (<= 1) results in very small numbers that can underflow, we sum the log probabilities instead.

In [14]:
import math

def log_probability(self, input_string):
    """ Returns the log-probability of the input string."""
    # example: [the, cat, sat, on, the, mat]
    # p(example) = p(sat | the cat) * p(on | cat sat) * p(the | sat on) * p(mat | on the)
    assert len(input_string) > 0, "input_string must be a non-empty string"

    # 1st step: apply the same preprocessing to the input string as we did to the training data
    words = [word.lower() for word in nltk.word_tokenize(input_string)]

    # 2nd step: calculate the log-probability
    log_probability = 0
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        ngram_minus_one = ngram[:-1]

        ngram_count = self.ngrams[ngram]
        ngram_minus_one_count = self.ngrams[ngram_minus_one]

        if ngram_minus_one_count == 0 or ngram_count == 0:
            log_probability += 0
            continue

        log_probability += math.log(ngram_count / ngram_minus_one_count)

    return log_probability

NGramLanguageModel.log_probability = log_probability

print(model.log_probability("How are you"))
print(model.log_probability("How are you in this longer sentence"))
print(model.log_probability("He she weird"))
print(model.log_probability("There is a house in New Orleans."))

-0.6931471805599453
-3.295836866004329
0
-2.0402208285265546


In [15]:
def inverse_log_probability(log_probability):
    return math.e ** log_probability

print(inverse_log_probability(model.log_probability("How are you")))
print(inverse_log_probability(model.log_probability("How are you in this longer sentence")))
print(inverse_log_probability(model.log_probability("He she weird")))
print(inverse_log_probability(model.log_probability("There is a house in New Orleans.")))

0.5
0.03703703703703704
1.0
0.13


Shorter texts will have higher log probability than longer texts, so we need to normalize it by the number of words in the input string.

In [16]:
def log_probability_normalized(self, input_string):
    """ Returns the log-probability of the input string."""
    # example: [the, cat, sat, on, the, mat]
    # p(example) = p(sat | the cat) * p(on | cat sat) * p(the | sat on) * p(mat | on the)
    assert len(input_string) > 0, "input_string must be a non-empty string"

    # 1st step: apply the same preprocessing to the input string as we did to the training data
    words = [word.lower() for word in nltk.word_tokenize(input_string)]

    # 2nd step: calculate the log-probability
    normalizing_count = 0
    log_probability = 0
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        ngram_minus_one = ngram[:-1]

        ngram_count = self.ngrams[ngram]
        ngram_minus_one_count = self.ngrams[ngram_minus_one]

        if ngram_minus_one_count == 0 or ngram_count == 0:
            log_probability += 0
            continue

        log_probability += math.log(ngram_count / ngram_minus_one_count)
        normalizing_count += 1 # only increase this if we really have an existing ngram

    return 0 if log_probability == 0 else log_probability / normalizing_count

NGramLanguageModel.log_probability = log_probability_normalized

print(inverse_log_probability(model.log_probability("How are you")))
print(inverse_log_probability(model.log_probability("How are you in this longer sentence")))
print(inverse_log_probability(model.log_probability("He she weird")))
print(model.log_probability("There is a house in New Orleans."))

0.5
0.19245008972987526
1.0
-2.0402208285265546


Lets predict the probabilities of two novels under our trained model: Jane Austen's *Sense and Sensibility* (`austen-sense.txt`) and Shakespeare's *Hamlet* (`shakespeare-hamlet.txt`).
- What do you expect will happen?
- What do you observe?

In [17]:
austen_sense_and_sensibility = gutenberg.raw("austen-sense.txt")
shakespeare_hamlet = gutenberg.raw("shakespeare-hamlet.txt")

print(model.log_probability(austen_sense_and_sensibility))
print(model.log_probability(shakespeare_hamlet))

-2.405643228181647
-2.7892762868035446


How many n-grams are known in each input?

In [ ]:
# keep track of the known ngrams in the log_probability function

10113
8219


Let's add a method `generate` that takes the start of a sentence ("prompt") and a number of words to generate, then continues our prompt.

In [ ]:
def generate(self, prompt, num_words=10):
    """ Continues a text starting with `prompt` for the `num_words` next words. """
    # 1) predict probability of next word
    # 2) take word with highest proba
    

NGramLanguageModel.generate = generate

Play around with a few different prompts.

In [11]:
print(model.generate("While she was sleeping, he was"))

None
